## Naver band web crawling
> https://band.us/keyword-group/%EB%82%98%EC%9D%B4%2F%EB%98%90%EB%9E%98%EB%AA%A8%EC%9E%84
- 네이버 벤드 주제별 제목과 설명 스크래핑
- 그룹: 나이/또래모임
- 소그룹: 40대, 50대, 60대
- 스크래핑 대상: 벤드 제목, 벤드 설명
- 제한조건: 40대 이상의 벤드를 가입해서 대화내용을 스크래핑을 하려고 했지만, 가입이 불가했다.
- 스크래핑 로직: URL접속 -> '40대' 클릭 -> 벤드명과 벤드명의 설명 스크래핑 -> '50대' 클릭 -> 반복

In [38]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import re
from tqdm import tqdm
import pandas as pd

driver = webdriver.Chrome('C:/Temp/chromedriver')
driver.implicitly_wait(3)

In [51]:
# 직접 로그인 후 진행
url = 'https://band.us/keyword-group/%EB%82%98%EC%9D%B4%2F%EB%98%90%EB%9E%98%EB%AA%A8%EC%9E%84'
driver.get(url)

# 수집할 List
title = []
subtitle = []

# '40대','50대','60대' 선택자
forties = '#content > div > div.keywordPageHead.fullWide > section > div > div > div.keywordMenuWrap._keywordContainer.gTaC > ul > li:nth-child(6) > a'
fifties = '#content > div > div.keywordPageHead.fullWide > section > div > div > div.keywordMenuWrap._keywordContainer.gTaC > ul > li:nth-child(7) > a'
sixties = '#content > div > div.keywordPageHead.fullWide > section > div > div > div.keywordMenuWrap._keywordContainer.gTaC > ul > li:nth-child(8) > a'

ages = [forties, fifties, sixties]
for age in tqdm(ages):

    # '40대','50대','60대' 클릭
    elem = driver.find_element_by_css_selector(age)
    elem.click()

    # 스크롤 무빙
    SCROLL_PAUSE_TIME = 2  # 스크롤 로딩 시간설정
    last_height = driver.execute_script("return document.body.scrollHeight") # 스크롤 높이
    count = 0

    for scroll in range(10):
        count += 1
        # 스크롤 무빙 
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 페이지 로드 대기 
        time.sleep(SCROLL_PAUSE_TIME) 
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);") # 맨 마지막까지 스크롤하면 바로 아래 리스트가 뜨지 않는 경우가 있기 때문에 -50 추가
        time.sleep(SCROLL_PAUSE_TIME)

        # 스크롤이 더이상 변화가 없으면 스크롤 이동 중단
        new_height = driver.execute_script("return document.body.scrollHeight") 
        if last_height == new_height:
            break
        last_height = new_height
    time.sleep(1)

    # 제목 수집
    title_selector = '#content > div > div.wide > section > ul > li > div > div.bandName > div'
    elems = driver.find_elements_by_css_selector(title_selector)
    for i in elems:
        title.append(i.text)
    time.sleep(1)

    # 소제목 수집
    subtitle_selector = '#content > div > div.wide > section > ul > li > div > div.bandName > p.pSubTxt.-multiLine'
    elems = driver.find_elements_by_css_selector(subtitle_selector)
    for i in elems:
        subtitle.append(i.text)
    time.sleep(1)

# title + subtitle
title_subtitle = title + subtitle
# 저장 (csv)
df = pd.DataFrame({'title_subtitle':title_subtitle})
df.to_csv('naver_band_40대~60대.csv', index = None)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:11<00:00, 23.72s/it]


## Preprocessing

- 형태소 분석기 사용하여 단어 빈출도 추출

In [1]:
import pandas as pd
import re

In [2]:
# 파일 불러오기
data = pd.read_csv('naver_band_40대~60대.csv')
data.drop_duplicates() # 중복행 제거

,title_subtitle
0,4050.♥️좋은사람.좋은인연♥️
1,♥4050 마주사랑™♥
2,"♥️4560행복하기 (설,경기)♥️"
3,4050부천인천 ❤ 불타는 청춘❤
4,40.50대 내생에 마지막 멋진삶
...,...
1701,"남미, 중미, 아프리카, 코카서스, 인도, 아이슬란드, 중동 지역 등의 특수지역을 ..."
1702,1970년대 동막골 2558부대 2포대에서 군복무하신 선후배 전우들 만남의 장으로~...
1703,골프를 좋아하고 배려심 깊고 회원을 먼저 생각할수있는 넓은마음을가지고있는 사람이면 ...
1704,행복한 동반자 회원닐들께 소통할수있는5060 부울경 남 지역님들 공간을 만들었습니다...


In [3]:
from konlpy.tag import Okt
from collections import Counter

In [4]:
# title data 합치기 (제거: '\n')
message = ''
for item in data['title_subtitle']:
    message = message + re.sub('[^\w]', ' ', item) +''

# 정규화 (제거할 단어)
message = re.sub('밴드', '', message)
message = re.sub('모임', '', message)
message = re.sub('가입', '', message)
message = re.sub('부산', '', message)
message = re.sub('지역', '', message)
message = re.sub('서울', '', message)
message = re.sub('경기', '', message)
message = re.sub('경기', '', message)

# message

In [5]:
# 명사 추출
nlp = Okt()
message_N = nlp.nouns(message)
# message_N

In [6]:
# 단어 빈도 확인
count = Counter(message_N)
word_count = dict()

for tag, counts in count.most_common(200):
    if(len(str(tag))>1):
        word_count[tag] = counts
        print(tag,':',counts)

친구 : 267
친목 : 183
사랑 : 134
인연 : 115
중년 : 112
환영 : 105
우리 : 99
사람 : 98
여행 : 96
골프 : 95
사진 : 86
인천 : 81
공간 : 81
대구 : 78
회원 : 70
힐링 : 68
인생 : 68
경남 : 67
추억 : 65
우정 : 62
누구 : 59
소통 : 57
이야기 : 55
나이 : 53
서로 : 53
울산 : 51
마음 : 51
전국 : 49
등산 : 48
트레킹 : 48
수원 : 47
부천 : 45
본인 : 45
사랑방 : 42
공유 : 42
산악회 : 42
이름 : 42
초등학교 : 41
광주 : 40
동행 : 39
행복 : 39
가족 : 39
거주 : 38
경북 : 35
일상 : 35
산행 : 34
필수 : 34
승인 : 33
활동 : 32
안양 : 31
안산 : 31
맛집 : 31
이상 : 29
대화 : 29
분만 : 29
졸업 : 28
기재 : 28
용인 : 27
대전 : 27
만들기 : 27
음악 : 27
실명 : 27
강남 : 26
분당 : 26
걷기 : 26
신청 : 26
영화 : 25
청춘 : 24
천안 : 24
건강 : 24
배려 : 24
시간 : 24
강퇴 : 24
충청 : 23
둘레길 : 23
도모 : 23
성남 : 22
일산 : 22
시니어 : 22
위해 : 22
참여 : 22
끼리 : 21
평택 : 21
리더 : 21
표기 : 21
창원 : 20
남녀 : 20
여기 : 20
취미 : 20
한잔 : 20
싱글 : 20
생각 : 20
초보 : 19
사모 : 19
토끼 : 19
매너 : 19
저희 : 19
화성 : 18
동창회 : 18
동창 : 18
정보 : 18
여성 : 18
초대 : 18
신분 : 18
부부 : 17
볼링 : 17
아산 : 17
오산 : 17
김해 : 17
운동 : 17
매월 : 17
목적 : 17
운영 : 17
사시 : 17
어울림 : 16
전북 : 16
말띠 : 16
시흥 : 16
의정부 : 16
뱀띠 : 16
전남 : 16
남양주 : 16
프샤 : 

In [7]:
import matplotlib.pyplot as plt

font_path = "../data/THEdog.ttf"   #폰트파일의 위치
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

plt.figure(figsize=(28,6))
plt.xlabel('키워드')
plt.ylabel('빈도수')
plt.grid(True)

sorted_Keys = sorted(word_count, key=word_count.get, reverse=True)
sorted_Values = sorted(word_count.values(), reverse=True)

plt.bar(range(len(word_count)), sorted_Values, align='center')
plt.xticks(range(len(word_count)), list(sorted_Keys), rotation='70')

plt.show()

NameError: name 'font_manager' is not defined

In [ ]:
from wordcloud import WordCloud

wc = WordCloud(font_path, background_color='white', width=800, height=600)
cloud=wc.generate_from_frequencies(word_count)

plt.figure(figsize=(15,8))
plt.imshow(cloud)
plt.axis('off')
plt.show()

# 워드클라우드 png저장
wc.to_file('naver_band_wordcloud.png')

# 워드 클라우드 색 찾아보기: scale_color_gradient(low = "#66aaf2", high = "#004EA1") +